In [1]:
import numpy as np

In [ ]:
# 윈도우 = keras 앞부분에 tensorflow를 붙여줘야한다.
# 우분투 = 앞에 붙일 필요 없이 keras만 작성해도 된다.
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
classifier = VGG16()

In [ ]:
# 마지막 레이어를 제거하여 추가적인 네트워크 구성이 가능하게 만든다.
last_layer = str(classifier.layers[-1])

In [ ]:
# 윈도우는 앞에 tensorflow. 붙이기 / 우분투는 tensorflow. 없어도 된다.
import tensorflow.keras

from tensorflow.keras.layers import Dense

In [ ]:
# 추가할 신규 레이어를 작성하는 파트
new_layer = tensorflow.keras.Sequential()

for layer in classifier.layers:
    if str(layer) != last_layer:
        new_layer.add(layer)
        
new_layer.summary()

In [ ]:
# 네트워크를 동결시키도록 한다.
for layer in new_layer.layers:
    layer.trainable = False

In [ ]:
new_layer.add(
    Dense(256, activation = 'relu')
)
new_layer.add(
    Dense(256, activation = 'relu')
)
new_layer.add(
    Dense(256, activation = 'relu')
)
new_layer.add(
    Dense(128, activation = 'relu')
)
new_layer.add(
    Dense(128, activation = 'relu')
)
new_layer.add(
    Dense(128, activation = 'sigmoid')
)
new_layer.add(
    Dense(64, activation = 'relu')
)
new_layer.add(
    Dense(32, activation = 'relu')
)
new_layer.add(
    Dense(16, activation = 'relu')
)
new_layer.add(
    Dense(8, activation = 'sigmoid')
)
new_layer.add(
    Dense(1, activation = 'sigmoid')
)

new_layer.summary()

In [ ]:
new_layer.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
# 윈도우 [tensorflow.] 추가
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
resModel = ResNet50()
resModel.summary()

In [ ]:
# 라플라스 변환을 하면 s-domain이라는 평면으로 이동을 하게 되고
# 라플라스 역변환을 하면 time-domaim(시간축)으로 돌아오게 된다.
from tensorflow.keras.applications.vgg16 import decode_predictions

In [ ]:
# 여기에 flask 모델 추가하는곳으로 예상
# vue에서 flask, axios로 python으로 이미지를 받아온다.

In [ ]:
# import socket

# (host, port) = ('localhost', 37373)

# s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# s.bind((host, port))
# s.listen(1)

# conn, addr = s.accept()

# with open('output', 'wb') as f:
# 	while True:
# 		l = conn.recv(1024)
# 		if not 1:
# 			break
# 		f.write(l)

# s.close()


In [ ]:
# 여기에 flask 모델 추가하는곳으로 예상
# vue에서 flask, axios로 python으로 이미지를 받아온다.
from flask import send_file
from flask import Flask
from flask_cors import CORS, cross_origin

# 분석 결과가 나왔으면 나온 결과를 vue로 연결해서 보여준다.
app = Flask(__name__)

CORS(app, resources={r'*': {'origins': 'http://localhost:8080'}})
@app.route('/img', methods = ['POST'])
def analysis_img():
    print('b')
    nickname = request.get_json()
    print('a')
    data = {"success": False}
    if request.method == 'POST':
        print('POST')
        print(nickname)
        print(nickname['nick'])
        name = nickname['nick']
        gra=rank(name)
        print(gra)
        return jsonify(gra)
    
if __name__ == "__main__":
    app.run()

In [ ]:
# # 일단 테스트 용도로 다양한 모델을 검증해보자.
# # 말그대로의 test 모델들을 넣어보자.
# # 현재 이미지 15까지
# test_A_img = image.load_img("static/test_image_17.jpg", target_size = (224, 224))
# test_A_img

In [ ]:
def rank(file):
    test_A_img = image.load_img(file, target_size = (224, 224))
    
    trans_img = image.img_to_array(test_A_img)
    trans_img = np.expand_dims(trans_img, axis = 0)
    trans_img = preprocess_input(trans_img)
    
    pred = classifier.predict(trans_img)
    label = decode_predictions(pred)
    
    print('top 5!')
    for i in range(5):
        print('%16s (%.2f%%)' % (label[0][i][1], label[0][i][2] * 100))

rank()

In [ ]:
# trans_img = image.img_to_array(test_A_img)
# trans_img = np.expand_dims(trans_img, axis = 0)
# trans_img = preprocess_input(trans_img)


# print(trans_img.shape)

In [ ]:
# pred = classifier.predict(trans_img)

# # pred = resModel.predict(trans_img)

In [ ]:
# label = decode_predictions(pred)

# def rank():
#     print('top 5!')
#     for i in range(5):
#         print('%16s (%.2f%%)' % (label[0][i][1], label[0][i][2] * 100))
#     rank()
        
